In [1]:
import sys
import os
import json
import pickle


def slashChanger(someString):
	tmp = ''
	for char in someString:
		if char == '\\':
			tmp += '/'
		else:
			tmp +=char
	return tmp

class fileRegister(object):
	"""docstring for fileRegister"""
	def __init__(self, arg):
		super(fileRegister, self).__init__()
		self.arg = arg




class singleShot(object):
	"""docstring for singleShot"""
	def __init__(self, id, pics_dir, json_dir, crop_dir, full_dir, spec_dir):
		super(singleShot, self).__init__()
		self.id       = id
		self.pics_dir = pics_dir
		self.json_dir = json_dir
		self.crop_dir = crop_dir
		self.full_dir = full_dir
		self.spec_dir = spec_dir
		self.crop_img_dir = None

		self.plot_img_full = None
		self.plot_img_crop = None
		self.plot_img_spec = None


		file = open(self.json_dir, 'r', encoding = 'utf-8')
		self.jsonDic = json.loads(file.read())
		file.close()

def refreshFolder(directory):
	directory = slashChanger(directory)
	print(directory)
	try:
		fnames = [s for s in os.listdir(directory) if os.path.isfile(directory + '/' + s) and '.jpeg' in s]
	except:
		return []
	print(fnames)
	fnames.sort()
	Fnames = [directory + '/' + s for s in fnames]

	mainThing = []
	for fname in fnames:
		tmp_pic_dir = directory + '/' + fname
		ID = fname.split('.')[0]
		tmp_json_dir = directory + '/' + ID + '.json' 		if os.path.isfile(directory + '/' + ID + '.json') else None
		tmp_crop_dir = directory + '/' + ID + '_crop.csv' 	if os.path.isfile(directory + '/' + ID + '_crop.csv') else None
		tmp_full_dir = directory + '/' + ID + '_full.csv' 	if os.path.isfile(directory + '/' + ID + '_full.csv') else None
		tmp_spec_dir = directory + '/' + ID + '_spec_NDVI.csv' if os.path.isfile(directory + '/' + ID + '_spec_NDVI.csv') else None
		mainThing.append(singleShot(ID, tmp_pic_dir, tmp_json_dir, tmp_crop_dir, tmp_full_dir, tmp_spec_dir))
	
	try:
		os.mkdir(directory + "/meta")
	except:
		pass

	with open(directory + '/meta/about.meta', 'wb') as config_dictionary_file:
		pickle.dump(mainThing, config_dictionary_file)
	return mainThing

def readMdata(directory):
	if (os.path.isfile(directory + '/meta/about.meta')):
		try:
			with open(directory + '/meta/about.meta', 'rb') as config_dictionary_file:
				mainThing = pickle.load(config_dictionary_file)
			return mainThing
		except:
			return None
	return refreshFolder(directory)
# refreshFolder(sys.aasdrgv[1])

In [301]:
import sys
import os
import json
import pickle
import shutil

# class SingleShot(object):
#     """docstring for singleShot"""
#     def __init__(self, id, pics_dir, json_dir, crop_dir, full_dir, spec_dir):
#         super(singleShot, self).__init__()
#         self.id       = id
#         self.pics_dir = pics_dir
#         self.json_dir = json_dir
#         self.crop_dir = crop_dir
#         self.full_dir = full_dir
#         self.spec_dir = spec_dir
#         self.crop_img_dir = None

#         self.plot_img_full = None
#         self.plot_img_crop = None
#         self.plot_img_spec = None

#         if self.json_dir != None:
#             file = open(self.json_dir, 'r', encoding = 'utf-8')
#             self.jsonDic = json.loads(file.read())
#             file.close()
class SingleShot(object):
    """docstring for singleShot"""
    def __init__(self, id,
                 origRGB_dir, json_dir, rawBMPs_dirs, CSVfull_dir, 
                 CSVcrop_dir, CSVspec_dir, RGBwithPOV_dir, BMPwithPOV_dir):
        self.id       = id
        
        self.origRGB_dir      = origRGB_dir
        self.json_dir         = json_dir
        self.rawBMPs_dirs     = rawBMPs_dirs
        
        self.CSVcrop_dir    = CSVcrop_dir
        self.CSVfull_dir    = CSVfull_dir
        self.CSVspec_dir    = CSVspec_dir
        self.RGBwithPOV_dir = RGBwithPOV_dir
        self.BMPwithPOV_dir = BMPwithPOV_dir
        
        self.jsonDic = None
        if self.json_dir != None:
            if os.path.isfile(self.json_dir):
                file = open(self.json_dir, 'r', encoding = 'utf-8')
                self.jsonDic = json.loads(file.read())
                file.close()



In [314]:
class SetFormatter():
    def __init__(self, directory, typeDictionary = None):
        self.dir = directory
        self.Types = {}          #mostly associated with the downloading types
        self.usingTypes = {}     #mostly associated with the types you want to use in your work
        self.allFnames = {}
        self.setWhichTypes(typeDictionary)
        self.checkFiles()
        self.usingUniqueID_inSeperateBuckets = []
        self.numShots = 0
        self.shots = []
        
    def setWhichTypes(self, tmpdict):
        self.Types['origRGB']    = tmpdict['origRGB']    if ("origRGB"    in tmpdict) else False
        self.Types['json']       = tmpdict['json']       if ("json"       in tmpdict) else False
        self.Types['rawBMPs']    = tmpdict['rawBMPs']    if ("rawBMPs"    in tmpdict) else False
        self.Types['CSVfull']    = tmpdict['CSVfull']    if ("CSVfull"    in tmpdict) else False
        self.Types['CSVcrop']    = tmpdict['CSVcrop']    if ("CSVcrop"    in tmpdict) else False
        self.Types['CSVspec']    = tmpdict['CSVspec']    if ("CSVspec"    in tmpdict) else False
        self.Types['RGBwithPOV'] = tmpdict['RGBwithPOV'] if ("RGBwithPOV" in tmpdict) else False
        self.Types['BMPwithPOV'] = tmpdict['BMPwithPOV'] if ("BMPwithPOV" in tmpdict) else False
        
    def dir_origRGB(self, ID):
        os.path.join(directory, '{}.jpg'.format(ID))
    def dir_json(self, ID):
        os.path.join(directory, '{}.json'.format(ID))
    def dir_rawBMPs(self, ID, i_th):
        os.path.join(directory, 'rawBMPs', '{}_{}.bmp'.format(ID,i_th))
    def dir_CSVfull(self, ID):
        os.path.join(directory, 'webGen', '{}_full.csv'.format(ID))
    def dir_CSVcrop(self, ID):
        os.path.join(directory, 'webGen', '{}_crop.csv'.format(ID))
    def dir_CSVspec(self, ID):
        os.path.join(directory, 'webGen', '{}_spec_NDVI.csv'.format(ID))
    def dir_RGBwithPOV(self, ID):
        os.path.join(directory, 'webGen', '{}.jpeg'.format(ID))
    def dir_BMPwithPOV(self, ID):
        os.path.join(directory, 'webGen', '{}.png'.format(ID))
    
    def checkFiles(self):
        self.allFnames['origRGB']    = [s for s in os.listdir(self.dir) if os.path.isfile(os.path.join(self.dir, s)) and '.jpg' in s]
        self.allFnames['json']       = [s for s in os.listdir(self.dir) if os.path.isfile(os.path.join(self.dir, s)) and '.json' in s]
        self.allFnames['rawBMPs']    = [s for s in os.listdir(os.path.join(self.dir,'rawBMPs')) if os.path.isfile(os.path.join(self.dir,'rawBMPs',s)) and '.bmp'           in s]
        self.allFnames['CSVfull']    = [s for s in os.listdir(os.path.join(self.dir, 'webGen')) if os.path.isfile(os.path.join(self.dir, 'webGen',s)) and '_full.csv'      in s]
        self.allFnames['CSVcrop']    = [s for s in os.listdir(os.path.join(self.dir, 'webGen')) if os.path.isfile(os.path.join(self.dir, 'webGen',s)) and '_crop.csv'      in s]
        self.allFnames['CSVspec']    = [s for s in os.listdir(os.path.join(self.dir, 'webGen')) if os.path.isfile(os.path.join(self.dir, 'webGen',s)) and '_spec_NDVI.csv' in s]
        self.allFnames['RGBwithPOV'] = [s for s in os.listdir(os.path.join(self.dir, 'webGen')) if os.path.isfile(os.path.join(self.dir, 'webGen',s)) and '.jpeg'          in s]
        self.allFnames['BMPwithPOV'] = [s for s in os.listdir(os.path.join(self.dir, 'webGen')) if os.path.isfile(os.path.join(self.dir, 'webGen',s)) and '.png'           in s]
    
    def cleanFiles(self):
        for s in os.listdir(self.dir):
            ss = os.path.join(self.dir, s)

            if os.path.isfile(ss):
                if not (s in self.allFnames['origRGB'] or 
                        s in self.allFnames['json']):
                    print("removed file:", s)
                    os.remove(ss)
            if os.path.isdir(ss):
                if (not s == "rawBMPs") and (not s == "webGen"):
                    shutil.rmtree(ss)
                    print("removed directory:", s)
                    
        tmpdir = os.path.join(self.dir, 'rawBMPs')
        for s in os.listdir(tmpdir):
            ss = os.path.join(tmpdir, s)
            if os.path.isfile(ss):
                if not (s in self.allFnames['rawBMPs']):
                    print("removed file: rawBMPs/", s)
                    os.remove(ss)                
            if os.path.isdir(ss):
                shutil.rmtree(ss)
                print("removed directory: rawBMPs/", s) 
        
        tmpdir = os.path.join(self.dir, 'webGen')
        for s in os.listdir(tmpdir):
            ss = os.path.join(tmpdir, s)
            if os.path.isfile(ss):
                if not (s in self.allFnames['CSVfull'] or 
                        s in self.allFnames['CSVcrop'] or 
                        s in self.allFnames['CSVspec'] or 
                        s in self.allFnames['RGBwithPOV'] or 
                        s in self.allFnames['BMPwithPOV']):
                    print("removed file: webGen/", s)
                    os.remove(ss)                
            if os.path.isdir(ss):
                shutil.rmtree(ss)
                print("removed directory: webGen/", s) 
    def read_ids(self, tmpdict, display = False):
        self.usingTypes['origRGB']    = tmpdict['origRGB']    if ("origRGB"    in tmpdict) else False
        self.usingTypes['json']       = tmpdict['json']       if ("json"       in tmpdict) else False
        self.usingTypes['rawBMPs']    = tmpdict['rawBMPs']    if ("rawBMPs"    in tmpdict) else False
        self.usingTypes['CSVfull']    = tmpdict['CSVfull']    if ("CSVfull"    in tmpdict) else False
        self.usingTypes['CSVcrop']    = tmpdict['CSVcrop']    if ("CSVcrop"    in tmpdict) else False
        self.usingTypes['CSVspec']    = tmpdict['CSVspec']    if ("CSVspec"    in tmpdict) else False
        self.usingTypes['RGBwithPOV'] = tmpdict['RGBwithPOV'] if ("RGBwithPOV" in tmpdict) else False
        self.usingTypes['BMPwithPOV'] = tmpdict['BMPwithPOV'] if ("BMPwithPOV" in tmpdict) else False
        
        self.usingUniqueID_inSeperateBuckets = []  #unique IDs over all set files (in seperate in their file types)
        if self.usingTypes['origRGB']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('.')[0]  for fname in self.allFnames['origRGB']])
            print(("origRGB   :unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['json']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('.')[0]  for fname in self.allFnames['json']])
            print(("json      :unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['CSVfull']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('_')[0] + '_'+ fname.split('_')[1]  for fname in self.allFnames['CSVfull']])
            print(("CSVfull   :unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['CSVcrop']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('_')[0] + '_'+ fname.split('_')[1]  for fname in self.allFnames['CSVcrop']])
            print(("CSVcrop   :unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['CSVspec']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('_')[0] + '_'+ fname.split('_')[1]  for fname in self.allFnames['CSVspec']])
            print(("CSVspec   :unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['RGBwithPOV']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('.')[0]  for fname in self.allFnames['RGBwithPOV']])
            print(("RGBwithPOV:unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['BMPwithPOV']:
            self.usingUniqueID_inSeperateBuckets.append([fname.split('.')[0]  for fname in self.allFnames['BMPwithPOV']])
            print(("BMPwithPOV:unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
        if self.usingTypes['rawBMPs']:
            self.usingUniqueID_inSeperateBuckets.append(list(set([fname.split('_')[0] + '_'+ fname.split('_')[1]  for fname in self.allFnames['rawBMPs']])))
            print(("rawBMPs   :unique count: {}\n".format(len(self.usingUniqueID_inSeperateBuckets[-1])) if display else ''),  end='')
            #self.usingUniqueID_inSeperateBuckets.append([fname.split('.')[0]  for fname in self.allFnames['rawBMPs']])
            #since rawBMPs have multiple files in a single shot, and I want only ID's so I did following
        
        #now calculating unique IDs over all set of files
        print(("============================\n" if display else ''),  end='')

        self.usingUniqueIDs = []
        Tmp = len(self.usingUniqueID_inSeperateBuckets)
        for i in range(len(self.usingUniqueID_inSeperateBuckets[0])):
            tmpID = self.usingUniqueID_inSeperateBuckets[0][i]
            tmpbool = True
            for t in range(1,Tmp):
                tmpbool = tmpbool and (tmpID in self.usingUniqueID_inSeperateBuckets[t])
                if not tmpbool:
                    print(("- incomplete shot: {} has no {}\n".format(tmpID, list(tmpdict.keys())[t]) if display else ''),  end='')
                    break
            if tmpbool:
                self.usingUniqueIDs.append(tmpID)
           
        self.usingUniqueIDs.sort()
        self.numShots = len(self.usingUniqueIDs)
        print(("==============================================\nIn Given types, {} unique shots in this Set.\n".format(self.numShots) if display else ''),  end='')
        
        
        for i in range(self.numShots):
            tmpid       = self.usingUniqueIDs[i]
            tmp_dirorigRGB = os.path.join(self.dir,  '{}.jpg'.format(tmpid)) if self.usingTypes['origRGB']    else None
            tmp_dirjson    = os.path.join(self.dir, '{}.json'.format(tmpid)) if self.usingTypes['json']       else None
            tmp_dirrawBMPs = os.path.join(self.dir,'rawBMPs', '{}.jpeg'.format(tmpid)) if self.usingTypes['rawBMPs']    else None
            tmp_dirCSVfull = os.path.join(self.dir,'webGen',  '{}_full.csv'.format(tmpid)) if self.usingTypes['CSVfull']    else None
            tmp_dirCSVcrop = os.path.join(self.dir,'webGen',  '{}_crop.csv'.format(tmpid)) if self.usingTypes['CSVcrop']    else None
            tmp_dirCSVspec = os.path.join(self.dir,'webGen',  '{}_spec_NDVI.csv'.format(tmpid)) if self.usingTypes['CSVspec']    else None
            tmp_RGBwithPOV = os.path.join(self.dir,'webGen',  '{}.jpeg'.format(tmpid)) if self.usingTypes['RGBwithPOV'] else None
            tmp_BMPwithPOV = os.path.join(self.dir,'webGen',   '{}.png'.format(tmpid)) if self.usingTypes['BMPwithPOV'] else None
            self.shots.append(SingleShot(id             = tmpid,
                                         origRGB_dir    = tmp_dirorigRGB, 
                                         json_dir       = tmp_dirjson, 
                                         rawBMPs_dirs   = tmp_dirrawBMPs, 
                                         CSVfull_dir    = tmp_dirCSVfull, 
                                         CSVcrop_dir    = tmp_dirCSVcrop,
                                         CSVspec_dir    = tmp_dirCSVspec,
                                         RGBwithPOV_dir = tmp_RGBwithPOV,
                                         BMPwithPOV_dir = tmp_BMPwithPOV,
                                        ))
            
        
        

In [321]:
Types = {
    "origRGB"   : True,
    "json"      : True,
    "rawBMPs"   : True,
    "CSVfull"   : True,
    "CSVcrop"   : True,
    "CSVspec"   : True,
    "RGBwithPOV": True,
    "BMPwithPOV": True,
}
sf = SetFormatter('/home/garid/myGithub/spectrumCatcher/spectrumCatcher/tests/dname1221/13h_', Types)
sf.read_ids(Types, display=True)

origRGB   :unique count: 326
json      :unique count: 326
CSVfull   :unique count: 326
CSVcrop   :unique count: 326
CSVspec   :unique count: 326
RGBwithPOV:unique count: 326
BMPwithPOV:unique count: 326
rawBMPs   :unique count: 326
- incomplete shot: 20191221_111708 has no json
In Given types, 325 unique shots in this Set.


In [322]:
sf.shots[0].CSVspec_dir

'/home/garid/myGithub/spectrumCatcher/spectrumCatcher/tests/dname1221/13h_/webGen/20191221_111630_spec_NDVI.csv'

In [263]:
list(Types.keys())[]

['origRGB',
 'json',
 'rawBMPs',
 'CSVfull',
 'CSVcrop',
 'CSVspec',
 'RGBwithPOV',
 'BMPwithPOV']

In [196]:
sf.allFnames['origRGB'].sort()
sf.allFnames['origRGB']

['20191221_111632.jpg',
 '20191221_111708.jpg',
 '20191221_111710.jpg',
 '20191221_111713.jpg',
 '20191221_111715.jpg',
 '20191221_111718.jpg',
 '20191221_111720.jpg',
 '20191221_111723.jpg',
 '20191221_111725.jpg',
 '20191221_111728.jpg',
 '20191221_111730.jpg',
 '20191221_111733.jpg',
 '20191221_111735.jpg',
 '20191221_111738.jpg',
 '20191221_111740.jpg',
 '20191221_111743.jpg',
 '20191221_111745.jpg',
 '20191221_111748.jpg',
 '20191221_111751.jpg',
 '20191221_111753.jpg',
 '20191221_111756.jpg',
 '20191221_111758.jpg',
 '20191221_111801.jpg',
 '20191221_111803.jpg',
 '20191221_111806.jpg',
 '20191221_111808.jpg',
 '20191221_111811.jpg',
 '20191221_111813.jpg',
 '20191221_111816.jpg',
 '20191221_111818.jpg',
 '20191221_111821.jpg',
 '20191221_111823.jpg',
 '20191221_111826.jpg',
 '20191221_111828.jpg',
 '20191221_111831.jpg',
 '20191221_111833.jpg',
 '20191221_111836.jpg',
 '20191221_111841.jpg',
 '20191221_111844.jpg',
 '20191221_111846.jpg',
 '20191221_111849.jpg',
 '20191221_11185

In [188]:
type(a)

set

In [35]:
dir ='/home/garid/myGithub/spectrumCatcher/spectrumCatcher/tests/dname1221/13h_'

[s for s in os.listdir(dir) if os.path.isfile(os.path.join(dir, s)) and '.json' in s]

['20191221_112701.json',
 '20191221_112601.json',
 '20191221_111821.json',
 '20191221_112132.json',
 '20191221_112952.json',
 '20191221_112842.json',
 '20191221_113035.json',
 '20191221_112240.json',
 '20191221_113020.json',
 '20191221_111828.json',
 '20191221_112042.json',
 '20191221_113057.json',
 '20191221_112754.json',
 '20191221_112935.json',
 '20191221_112704.json',
 '20191221_112736.json',
 '20191221_112553.json',
 '20191221_112734.json',
 '20191221_112433.json',
 '20191221_112501.json',
 '20191221_113055.json',
 '20191221_112656.json',
 '20191221_113015.json',
 '20191221_112428.json',
 '20191221_112453.json',
 '20191221_112134.json',
 '20191221_112955.json',
 '20191221_112202.json',
 '20191221_112546.json',
 '20191221_112927.json',
 '20191221_112646.json',
 '20191221_113027.json',
 '20191221_112817.json',
 '20191221_112400.json',
 '20191221_112430.json',
 '20191221_112044.json',
 '20191221_112827.json',
 '20191221_111959.json',
 '20191221_112348.json',
 '20191221_112641.json',


In [80]:
import numpy as np
country = np.array(['USA', 'Japan', 'UK', '', 'India', 'China']) 

In [81]:
country

array(['USA', 'Japan', 'UK', '', 'India', 'China'], dtype='<U5')